In [ ]:
import math
import functools

rads = math.pi / 180
eRadius = 6371 #km

In [ ]:
def round(v, epislon=0.00001):
    if abs(v) < epislon:
        return 0
    if v + epislon > 1:
        return 1
    if v - epislon < -1:
        return -1
    return v

In [ ]:
def roundFloat(v, epislon=0.00001):
    sign = 1 if v > 0 else -1
    absV = abs(v)
    if int(absV) < int(absV + epislon):
        return sign * float(int(absV + epislon))
    return v

In [ ]:
#Convert lat/lon (must be in radians) to Cartesian coordinates for each location.
def toXYZ(latLon):
    rLat,rLon = latLon[0] * rads, latLon[1] *rads

    cLat = round(math.cos(rLat))

    X = cLat * round(math.cos(rLon))
    Y = cLat * round(math.sin(rLon))
    Z = round(math.sin(rLat))
    return [X,Y,Z]

In [ ]:
def checkPlane(plane,vector):
    check = 0
    for x in range(len(plane)):
        check += plane[x] * vector[x]
    return round(check) > 0

In [ ]:
def toLatLon(xyz):
    lat = math.atan2(xyz[2], (xyz[0] * xyz[0] + xyz[1] * xyz[1])**(1/2)) / rads
    lon = math.atan2(xyz[1],xyz[0]) / rads
    return [lat, lon]

In [ ]:
def haversine(start, stop):
    startLat = start[0] * rads
    startLon = start[1] * rads

    stopLat = stop[0] * rads
    stopLon = stop[1] * rads

    latDiff = stopLat - startLat
    lonDiff = stopLon - startLon

    sinLatDiff = math.sin(latDiff / 2)
    sinLonDiff = math.sin(lonDiff / 2)

    a = sinLatDiff * sinLatDiff + math.cos(startLat) * math.cos(stopLat) * sinLonDiff * sinLonDiff

    c = 2 * math.atan2( a**(1/2), (1-a)**(1/2) ) #angular distance
    return c * eRadius

    

In [ ]:
def bearing(start,stop):
    startLat = start[0] * rads
    startLon = start[1] * rads

    stopLat = stop[0] * rads
    stopLon = stop[1] * rads

    lonDiff = stopLon - startLon

    return math.atan2(
        math.sin(lonDiff) * math.cos(stopLat),
        math.cos(startLat) * math.sin(stopLat) - math.sin(startLat) * math.cos(stopLat) * math.cos(lonDiff)
    )


In [ ]:
def destination(latLon,d,b):
    lat = latLon[0] * rads
    lon = latLon[1] * rads

    a = d / eRadius

    lat2 = math.asin(math.sin(lat) * math.cos(a) + math.cos(lat) * math.sin(a) * math.cos(b))
    lon2 = lon + math.atan2(math.sin(b) * math.sin(a) * math.cos(lat), math.cos(a) - math.sin(lat) * math.sin(lat2))
    return [roundFloat(lat2 / rads), ( roundFloat(lon2 / rads) + 540) % 360 - 180]

In [ ]:
def toGnomonic(latLon, cLatLon):
    lat = latLon[0] * rads
    lon = latLon[1] * rads

    cLat = cLatLon[0] * rads
    cLon = cLatLon[1] * rads

    ccLat = math.cos(cLat)
    sLat = math.sin(lat)
    scLat = math.sin(cLat)
    cosLat = math.cos(lat)
    lonDelta = lon - cLon
    cosLonDelta = math.cos(lonDelta)

    cosc = scLat * sLat + ccLat * cosLat * cosLonDelta
    x = cosLat * math.sin(lonDelta) / cosc
    y = ( ccLat * sLat - scLat * cosLat * cosLonDelta ) / cosc

    return [x,y] 

In [ ]:
def fromGnomonic(xy, cLatLon):
    p = (xy[0] * xy[0] + xy[1] * xy[1]) ** (1/2)
    c = math.atan(p)

    cLat = cLatLon[0] * rads
    cLon = cLatLon[1] * rads

    cosc = math.cos(c)
    sLat = math.sin(cLat)
    sinc = math.sin(c)
    cosLat =  math.cos(cLat)

    lon = math.asin(cosc * sLat + xy[1] * sinc * cosLat / p) 
    lat = cLat + math.atan( xy[0] * sinc / ( p * cosLat * cosc - xy[1] * sLat * sinc ) ) 
    return [roundFloat(lat / rads) ,roundFloat(lon / rads)]

In [ ]:
def threeCross(a,b):
    return [ 
        a[1]*b[2] - a[2]*b[1],
        a[2]*b[0] - a[0]*b[2],
        a[0]*b[1] - a[1]*b[0]
    ]

In [ ]:
def split(start,stop):
    xyzStart = toXYZ(start)
    xyzStop = toXYZ(stop)
    startToStop = threeCross(xyzStart,xyzStop)
    startSplit = toXYZ([89.9,180])
    endSplit = toXYZ([-89.9,180])
    splitCross = threeCross(startSplit, endSplit)

    intersection = threeCross(startToStop, splitCross)
    intersection2 = threeCross(splitCross,startToStop)
    print(toLatLon(intersection))
    print(toLatLon(intersection2))
    #print(f"{startToStop} {startSplit} {endSplit} {splitCross} {intersection}")

In [ ]:
split([60,170],[60,-170])

In [ ]:
def convexHull(latLonArray):
    #convert to xyz
    #find center xyz
    #filter to xyz's above the plane created by origin to center vector
    #we need the lat lon points that pass this test
    #take lat,lons and gnomonic project based on center
    #hull these points
    #reproject hull points
    #slice into two groups for easier handling of anti-merdian

In [ ]:
def orientation(p,q,r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    val = round(val)
    if val == 0:
        return 0
    elif val > 0:
        return 1
    else: 
        return 2


In [ ]:
def sortCriteria(x,y,p):
    o = orientation(p,x,y)
    if o == 0:
        if p[0] * x[0] + p[1] * x[1] > p[0] * y[0] + p[1] * y[1]:
            return 1
        else:
            return -1
    else:
        if o == 2:
            return -1
        else:
            return 1

In [ ]:
def makeSortCriteria(p):
    return lambda x,y : sortCriteria(x,y,p)

In [ ]:
def grahamScan(xyArray):
    #find lowest point on y if tie take fatherest left
    pMix = xyArray[0]
    minSpot = 0
    n = len(xyArray)
    for i in range(1,n):
        if xyArray[i][1] < pMix[1] or xyArray[i][1] == pMix[1] and xyArray[i][0] < pMix[0]:
            minSpot = i
            pMix = xyArray[i]
    #now sort based on first orientation from first spot and then distance from first spot if tied
    compare = makeSortCriteria(pMix)     
    sorted(xyArray, key=functools.cmp_to_key(compare))
    m = 1
    stop = n -1
    i = 1 
    # go through and exclude points that have the same orientation
    while(i < n):
        while i < stop:
            o = orientation(pMix,xyArray[i],xyArray[i+1])
            if o == 0:
                i += 1
            else:
                break
        xyArray[m] = xyArray[i]
        i += 1
        m += 1
    if m < 3:
        return xyArray[0:m]

    s = []
    s.append(xyArray[0])
    s.append(xyArray[1])
    s.append(xyArray[2])
    sLen = len(s)
    for i in range(3,m):
        while ( (sLen > 1) and (orientation(s[sLen - 2], s[sLen -1 ], xyArray[i])) != 2):
            s.pop()
            sLen = sLen - 1
        s.append(xyArray[i])
        sLen = sLen + 1
    return s
    

In [ ]:
d = haversine([45,45],[46,47])
b = bearing([45,45],[46,47])
newP = destination([45,45], 191.461, 0.93876502)
print(f"distance {d} bearing {b} point {newP}")

In [ ]:
xy = toGnomonic([45,45],[60,60])
latLon = fromGnomonic(xy,[60,60])
print(f"lat {latLon[0]} lon {latLon[1]}")

In [ ]:
xyz = toXYZ([45,45])
lat,lon = toLatLon(xyz)
print(f"lat {lat} lon {lon}")

In [ ]:
plane = toXYZ([89.99999,0])
print(checkPlane(plane,toXYZ([50,-40])))

In [ ]:
p = [[1,2],[2,2],[5,2],[3,3],[4,4]]
grahamScan(p)